In [2]:
import pandas as pd

In [28]:
events = pd.read_excel("events.xlsx")
key_segmetns = pd.read_excel("key segments.xlsx")
segment_data = pd.read_csv("segment_data.csv", sep=";")
monday_speeds = pd.read_csv("monday-speeds.csv", sep=";")
segments_ids_nord_direction = pd.read_csv("north_direction_road_segments.txt", header=None, names=['SegmentId'])

### Event Document

In [29]:
events.head(1)

,Skapad_År,Skapad_Månad,NAME,EVENT_CODE,EVENT_TEXT,Location_text,Väg,Riktning,LANE_MASK,CREATION_TIME,COMPLETION_TIME,START_TIME,END_TIME,Note,X,Y
0,2019,Januari,2019-01-01/03:34:40-1/4,35,Olycka,E4 från Trafikplats Sörljusne (204) till Trafi...,E4,Syd,OOOOOOOOX|OOOOOOOOO,2019-01-01 03:34:40,2019-01-01 06:01:30,2019-01-01 03:34:40,2019-01-01 06:15:00,I höjd med Böles-Noran i södergående. En perso...,6774947,606946


### SegmentID direction and coordinates

In [30]:
key_segments.head(1)

,RouteID,SegmentID,ValidFrom,ValidTo,Längd,Färdriktn,Latitud,Longitud,Ordning,Väg,Antal kf,Hastighet,Korsning
0,1520100057,1071754891,2019-04-16,9999-12-31 00:00:00,930,Syd,"61,165335","16,851284",1,83,1kf,90,NaN


### SegmentId meta data

In [31]:
segment_data.head(1)

,SegmentId,Length,Road,Antal kf,Max_speed
0,1071754891,930,83,1kf,90


### SegmentID and speeds

In [32]:
monday_speeds.head(1)

,SegmentId,StartTime,EndTime,Speed
0,1071707139,2019-06-03 00:00:00,2019-06-03 00:30:00,NaN


# The Goal

- Create a deep learning model to predict traffic speed after a traffic event until it returns to normal speed

# What variables needed to do predictions?
- SegmentID
- Speed in time
- Events
    - Code/Text
    - CREATION_TIME 	
    - COMPLETION_TIME 	
    - START_TIME 	
    - END_TIME

- Weather data is not needed to do the project and therefore not included here. Most important is to link speed and events first and then we can expand with weather data

# Observations

#### Event Document
- The event code
- X and Y location (this is most likely not latitude and longitude)

#### SegmentID and Speed
- segmentID
- Speed

#### SegmentId and coordinates
- SegmentID
- Latitude and Longitude coordinates

# My take on it

It's unclear if Latitude and Longitude can be connected to the X,Y from the event document which is required to link SegmentID, Speed and Events.

<br>

Therefore, questions to answer
- What type of coordinate do the X,Y represent in the event document since they don't look like latitude and longitude?

In [6]:
df = pd.read_csv("../data/events-with-lat-long-segmentID.csv", sep=",")

In [7]:
this_segment = df[df['SegmentID'] == 226255131]

In [8]:
this_segment

,Unnamed: 0,Skapad_År,Skapad_Månad,NAME,EVENT_CODE,EVENT_TEXT,Location_text,Väg,Riktning,LANE_MASK,CREATION_TIME,COMPLETION_TIME,START_TIME,END_TIME,Note,X,Y,latitude,longitude,SegmentID
1338,1338,2021,Juli,2021-07-27/23:16:50-1/4,40,Vägarbete,E4 från Axmartavlan (202) till Trafikplats Tön...,E4,Norr,OOOOOOOOO|XOOOOOOOO,2021-07-27 23:16:50,2021-07-28 00:32:15,2021-07-27 23:16:50,2021-07-28 00:30:00,Underhållsarbete i höjd med Högbacka. Risk för...,6765123,607357,61.00636,16.985545,226255131


In [47]:
segment_ids = []
segment_ids_count = []

for segment_id in df['SegmentID'].values:
    
    
    counts = df['SegmentID'].value_counts()
    
    #print(counts)
    
    if segment_id not in segment_ids:
        
        
        count = counts.get(segment_id)
        
        print(segment_id, count)
        
        segment_ids_count = [str(segment_id), count]
        segment_ids.append(segment_id)
        

1071863723 8
428403354 110
428390707 3
1071723605 90
1071723800 3
1071723711 213
428389808 260
428390704 102
1071845849 3
428399460 36
428397749 5
1071729662 28
1071846041 70
1071832339 69
1071863882 42
1071707121 4
1071707139 3
1071707171 8
428389804 17
226992808 3
1071846229 12
428397748 19
1071832389 5
226910815 2
1071754109 39
226944388 6
1071845928 1
226954600 3
1071727508 6
428403353 1
1071891776 12
1071723838 3
1071727490 6
1071703530 15
428403369 16
1071723686 5
1071703593 5
428402118 3
1071754197 1
1071754891 45
428399463 1
1071863848 3
1071863900 3
374870394 3
1071727524 5
1071723815 4
1071723637 3
1071727542 3
375114290 2
1071723676 6
1071703625 8
428390708 1
375160804 6
226736042 5
1071860329 2
1071845991 2
1071845973 5
428403366 7
1071727530 48
428389807 3
1071845832 4
1071845863 8
1071790317 10
1071846073 5
1071863801 2
428397715 2
374986318 1
1071754151 1
1071729262 1
1071832566 6
1071729215 3
374928641 2
428389805 4
1071729231 1
1071863482 3
1071863817 3
428390709 3
107

We can see that segment_id: 428389808 has the most events